In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import yaml
from itertools import chain
import os
import random

In [2]:
def get_file_names(path):
    # get list of papers .json from path
    file_names = []
    for file_name in os.listdir(path):
        file_names.append(file_name)
    return file_names

In [3]:
def generate_sha(used=[]):
    hash = ''
    while (hash == 'nan') or (hash in used):
        hash = "%032x" % random.getrandbits(128)
    return hash

# Classes

- Paper => BiorxivPaper, (and others)
- Scraper => BiorxivScraper (and others)

In [4]:
class Paper:
    def __init__(self, source):
        self.source = source
        self.paper = {} # contains paper data
    
    def import_json(self, path):
        ''' If exists, saves JSON article to file'''
        if os.path.exists(path):
            with open(path) as json_file:
                self.paper = json.load(json_file)
            self.string = self.to_str()
            return True
        else:
            return False
    
    def save_json(self, path):
        ''' Import paper from json file '''
        if self.paper and os.path.exists(path):
            with (path, 'w') as outfile:
                json.dump(self.paper, outfile)
            return True
        else:
            return False
    
    def to_html(self):
        ''' Convert article to HTML '''
        return '<html></html>'
    
    def save_html(self, path):
        ''' If exists, saves HTML article to file '''
        return True
    
    def ref_to_str(self, reference):
        ''' Convert reference to string '''
        output = ''
        if reference['label'] != '':
            output = reference['label'] + ' - '
        if reference['title'] != '':
            output += reference['title'] + ', '
        if reference['authors'] != []:
            output += ', '.join(reference['authors'])
        if reference['date'] != None:
            output += reference['date']
        return output

    def graph_to_str(self, graph, indent=''):
        ''' Converts a graphic element: table or figure to string '''
        output = '\n'
        output += indent + '-'*100 + '\n'
        output += indent + graph['id'] + ' | ' + graph['label'] + '\n'
        output += indent + graph['img'] + '\n'
        output += indent + graph['description']
        output += indent + '-'*100
        output += '\n'
        return output
    
    def section_to_str(self, section, depth=0):
        indent = '  '*depth
        output = indent + section['title']
        for content in section['content']:
            if type(content) == dict:
                if 'title' in content.keys():
                    # is section = explore further 
                    output += '\n'
                    output += self.section_to_str(content, depth+1)
                elif 'id' in content.keys():
                    # is fig or table, get string
                    if content['type'] == 'table':
                        output += '\n'
                        output += self.graph_to_str(self.paper['tables'][content['id']], indent) # content['id']
                        # insert table description from paper['tables']
                    elif content['type'] == 'figure':
                        output += ' \n'
                        output += self.graph_to_str(self.paper['figures'][content['id']], indent) # content['id']
                        # insert figure description from paper['figures']
                    else:
                        print('content not recognized: {}'.format(content))
            elif type(content) == str:
                output += '\n' + indent + content
        return output
    
    def to_str(self, appendix=False):
        ''' Convert article to string '''
        ''' Converts a paper to string formart '''
        output = '({} | {} | {})\n'.format(self.paper['source'], self.paper['id'], self.paper['url'])
        output += self.paper['title'] + '\n'
        output += ', '.join(self.paper['authors']) + '\n'
        for section in self.paper['sections']:
            if (not appendix) and ('appendix' in section['title'].lower()):
                pass
            else:
                output += self.section_to_str(section)
                output += '\n'
        output += '\n\n'.join([self.ref_to_str(ref) for ref in self.paper['references']])    
        return output

In [ ]:
class BiorxivScraper():
    def __init__(self, papers):
        self.source = 'biorxiv'
        self.session = requests.session()
        self.base_url = ''
    
    def get_paper_soup(self, url):
        ''' Get paper soup from source '''
        html = self.session.get(url)
        self.soup = BeautifulSoup(html.content, 'html.parser')
    
    def paper_from_soup(self):
        ''' Extract paper data from soup '''
        return 
    
    def get_authors(authors_html):
        ''' HTML list of authors '''
        authors = []
        for author in authors_html:
            try:
                given_name = author.find('span',attrs={'class':'nlm-given-names'}).text
            except:
                given_name = ''
            try:
                surname = author.find('span',attrs={'class':'nlm-surname'}).text
            except:
                surname = ''
            authors.append(given_name + ' ' + surname)
        return authors
    
    def get_tables(html_tables, base_url):
        ''' Get tables from HTML '''
        tables = {}
        for html_table in html_tables:
            table = {}

            table['id'] = html_table['id']
            table_label = html_table.find('span',attrs={'class':'table-label'})
            if table_label:
                table['label'] = table_label.text

            table_caption = html_table.find('span',attrs={'class':'caption-title'})
            if table_caption:
                table['caption'] = table_caption.text

            table['description'] = ''
            for p in html_table.find_all('p'):
                table['description'] += p.text + '\n'
            table['img'] = '{}/{}.large.jpg'.format(base_url,table['id'])
            tables[table['id']] = table
        return tables
    
    def get_biorxiv_figures(figures_html):
        ''' Get figures from HTML '''
        figures = {}
        for html_figure in figures_html:
            figure = {}
            figure['id'] = html_figure['id']
            figure_label = html_figure.find('span',attrs={'class':'fig-label'})
            if figure_label:
                figure['label'] = figure_label.text

            figure_title = html_figure.find('span',attrs={'class':'caption-title'})
            if figure_title:
                figure['title'] = figure_title.text

            figure['description'] = ''
            figure_ps = html_figure.find_all('p')
            for p in figure_ps:
                figure['description'] += p.text + '\n'
            figure_img = html_figure.find('a')
            if figure_img:
                figure['img'] = figure_img['href'].split('?')[0]
            figures[figure['id']] = figure
        return figures
    
    def process_paragraph(paragraph):
        ''' Removes and modifies html tags in paragraphs '''
        for anchor in paragraph.find_all('a'):
            anchor.replace_with('[' + anchor['href'].strip('#') + ',"' + anchor.text + '"]')
        return paragraph.text
    
    def explore_section(root_tag):
        ''' Recursively explore section and all subsections for content '''
        content = {}
        content['title'] = ''
        content['content'] = []
        for child in root_tag.findChildren(recursive=False):
            if child.name == 'h2' or child.name == 'h3' or child.name == 'h4':
                content['title'] = child.text
            elif child.name == 'p':
                content['content'].append(process_paragraph(child))
            if child.name == 'div':
                if 'class' in child.attrs:
                    if 'subsection' in child['class']:
                        # if subsection explore recursively
                        content['content'].append(explore_section(child))
                if 'id' in child.attrs.keys():
                    if 'T' in child['id']:
                        content['content'].append({'id':child['id'],'type':'table'})
                    elif 'F' in child['id']:
                        content['content'].append({'id':child['id'],'type':'figure'})
        else:
            return content
    
    def get_references(references_html):
        ''' HTML list of references '''
        references = []
        for ref in references_html:
            reference = {}
            # label
            try:
                reference['label'] = ref.find('span',attrs={'class':'ref-label'}).text
            except:
                reference['label'] = None

            # authors
            author_names = []
            try:
                authors = ref.find('cite').find_all('span',attrs={'class':'cit-auth'})
                for author in authors:
                    surname = author.find('span',attrs={'class':'cit-name-surname'}).text
                    given_names = author.find('span',attrs={'class':'cit-name-given-names'}).text
                    author_name = surname + ' ' + given_names
                    author_names.append(author_name)
                reference['authors'] = author_names
            except:
                reference['authors'] = author_names

            # date
            try:
                reference['date'] = ref.find('cite').find('span',attrs={'class':'cit-pub-date'}).text
            except:
                reference['date'] = None

            # title
            try:
                reference['title'] = ref.find('cite').find('span',attrs={'class':'cit-article-title'}).text
            except:
                reference['title'] = ''

            # links
            ref_links = []
            try:
                links = ref.find('div',attrs={'class':'cit-extra'}).find_all('a')
                for link in links:
                    if link.text != 'OpenUrl':
                        link_name = link.text
                        link_url = link['href']
                        ref_links.append((link_name,link_url))
                reference['links'] = ref_links
            except:
                reference['links'] = ref_links
            references.append(reference)
        return references
    
    def get_data_from_soup(self):
        '''
            Extract paper information from html webpage of biorxiv
        '''
        content_url = self.soup.find('meta',attrs={'name':'citation_pdf_url'})['content'].strip('.full.pdf')

        paper = {}
        paper['title'] = self.soup.h1.text

        authors_html = self.soup.find('span',attrs={'class':'highwire-citation-authors'}) \
                           .find_all('span',attrs={'class':'highwire-citation-author'})
        paper['authors'] = get_authors(authors_html)

        sections_html = self.soup.find_all('div',attrs={'class':'section'})
        paper['sections'] = get_sections(sections_html)

        tables_html = self.soup.find_all('div',attrs={'class':'table'})
        paper['tables'] = get_tables(tables_html, content_url)

        # get figures
        figures_html = self.soup.find_all('div',attrs={'class':'fig'})


        paper['figures'] = get_figures(figures_html)

        # get references
        references_html = self.soup.find('div',attrs={'class':'section ref-list'}).find_all('li')
        paper['references'] = get_references(references_html)

        return paper
    
    def scrape_paper(self, url):
        self.get_paper_soup()

In [244]:
test_paper = Paper('biorxiv')

In [246]:
test_paper.import_json('data/biorxiv/01e3b313e78a352593be2ff64927192af66619b5.json')

TypeError: string indices must be integers

In [247]:
test_paper.paper

{'source': 'biorxiv',
 'id': '01e3b313e78a352593be2ff64927192af66619b5',
 'url': 'https://biorxiv.org/content/10.1101/029397v1.full',
 'title': 'Viruses are a dominant driver of protein adaptation in mammals',
 'authors': ['David Enard', 'Le Cai', 'Carina Gwenapp', 'Dmitri A. Petrov'],
 'sections': {'Summary': 'Viruses interact with hundreds to thousands of proteins in mammals, yet adaptation against viruses has only been studied in a few proteins specialized in antiviral defense. Whether adaptation to viruses typically involves only specialized antiviral proteins or affects a broad array of proteins is unknown. Here, we analyze adaptation in ~1,300 virus-interacting proteins manually curated from a set of 9,900 proteins conserved across mammals. We show that viruses (i) use the more evolutionarily constrained proteins from the cellular functions they hijack and that (ii) despite this high constraint, virus-interacting proteins account for a high proportion of all protein adaptation in

# Functions

## Scraping functions

In [252]:
def scrape_biorxiv(paper):
    print('Fetching paper data at {}'.format(paper['url']))
    html = session.get(paper['url'])
    tmp_soup = BeautifulSoup(html.content, 'html.parser')
    paper_webdata = biorxiv_html_paper_data_extractor(tmp_soup)
    paper = dict(chain(paper.items(),paper_webdata.items()))
    return paper

In [253]:
def get_biorxiv_authors(authors_html):
    ''' HTML list of authors
    '''
    authors = []
    for author in authors_html:
        try:
            given_name = author.find('span',attrs={'class':'nlm-given-names'}).text
        except:
            given_name = ''
        try:
            surname = author.find('span',attrs={'class':'nlm-surname'}).text
        except:
            surname = ''
        authors.append(given_name + ' ' + surname)
    return authors

In [254]:
# get reference anchor id
def get_biorxiv_references(references_html):
    ''' HTML list of references
    soup.find('div',attrs={'class':'section ref-list'}).find_all('li')
    '''
    references = []
    for ref in references_html:
        reference = {}
        # label
        try:
            reference['label'] = ref.find('span',attrs={'class':'ref-label'}).text
        except:
            reference['label'] = None

        # authors
        author_names = []
        try:
            authors = ref.find('cite').find_all('span',attrs={'class':'cit-auth'})
            for author in authors:
                surname = author.find('span',attrs={'class':'cit-name-surname'}).text
                given_names = author.find('span',attrs={'class':'cit-name-given-names'}).text
                author_name = surname + ' ' + given_names
                author_names.append(author_name)
            reference['authors'] = author_names
        except:
            reference['authors'] = author_names

        # date
        try:
            reference['date'] = ref.find('cite').find('span',attrs={'class':'cit-pub-date'}).text
        except:
            reference['date'] = None

        # title
        try:
            reference['title'] = ref.find('cite').find('span',attrs={'class':'cit-article-title'}).text
        except:
            reference['title'] = ''

        # links
        ref_links = []
        try:
            links = ref.find('div',attrs={'class':'cit-extra'}).find_all('a')
            for link in links:
                if link.text != 'OpenUrl':
                    link_name = link.text
                    link_url = link['href']
                    ref_links.append((link_name,link_url))
            reference['links'] = ref_links
        except:
            reference['links'] = ref_links
        references.append(reference)
    return references

In [255]:
def get_biorxiv_sections(sections_html):
    '''
    '''
    sections = []
    for section in sections_html:
        # ignore references
        if 'ref-list' not in section.attrs['class'] :
            sections.append(explore_section(section))
    return sections

In [256]:
# make tables a dict with table id for easy searching
def get_biorxiv_tables(html_tables, base_url):
    tables = {}
    for html_table in html_tables:
        table = {}
        
        table['id'] = html_table['id']
        table_label = html_table.find('span',attrs={'class':'table-label'})
        if table_label:
            table['label'] = table_label.text

        table_caption = html_table.find('span',attrs={'class':'caption-title'})
        if table_caption:
            table['caption'] = table_caption.text

        table['description'] = ''
        for p in html_table.find_all('p'):
            table['description'] += p.text + '\n'
        table['img'] = '{}/{}.large.jpg'.format(base_url,table['id'])
        tables[table['id']] = table
    return tables

In [257]:
# make figures a dict with figure id for easy searching
def get_biorxiv_figures(figures_html):
    figures = {}
    for html_figure in figures_html:
        figure = {}
        figure['id'] = html_figure['id']
        figure_label = html_figure.find('span',attrs={'class':'fig-label'})
        if figure_label:
            figure['label'] = figure_label.text

        figure_title = html_figure.find('span',attrs={'class':'caption-title'})
        if figure_title:
            figure['title'] = figure_title.text

        figure['description'] = ''
        figure_ps = html_figure.find_all('p')
        for p in figure_ps:
            figure['description'] += p.text + '\n'
        figure_img = html_figure.find('a')
        if figure_img:
            figure['img'] = figure_img['href'].split('?')[0]
        figures[figure['id']] = figure
    return figures

In [258]:
def process_paragraph(paragraph):
    ''' Removes and modifies html tags in paragraphs
    '''
    for anchor in paragraph.find_all('a'):
        anchor.replace_with('[' + anchor['href'].strip('#') + ',"' + anchor.text + '"]')
    return paragraph.text

In [259]:
def explore_section(root_tag):
    '''Recursively explore section and all subsections for content
    '''
    content = {}
    content['title'] = ''
    content['content'] = []
    for child in root_tag.findChildren(recursive=False):
        if child.name == 'h2' or child.name == 'h3' or child.name == 'h4':
            content['title'] = child.text
        elif child.name == 'p':
            content['content'].append(process_paragraph(child))
        if child.name == 'div':
            if 'class' in child.attrs:
                if 'subsection' in child['class']:
                    # if subsection explore recursively
                    content['content'].append(explore_section(child))
            if 'id' in child.attrs.keys():
                if 'T' in child['id']:
                    content['content'].append({'id':child['id'],'type':'table'})
                elif 'F' in child['id']:
                    content['content'].append({'id':child['id'],'type':'figure'})
    else:
        return content

In [260]:
def biorxiv_html_paper_data_extractor(soup):
    '''
        Extract paper information from html webpage of biorxiv
    '''
    content_url = soup.find('meta',attrs={'name':'citation_pdf_url'})['content'].strip('.full.pdf')
    
    paper = {}
    paper['title'] = soup.h1.text

    authors_html = soup.find('span',attrs={'class':'highwire-citation-authors'}) \
                       .find_all('span',attrs={'class':'highwire-citation-author'})
    paper['authors'] = get_biorxiv_authors(authors_html)

    sections_html = soup.find_all('div',attrs={'class':'section'})
    paper['sections'] = get_biorxiv_sections(sections_html)

    tables_html = soup.find_all('div',attrs={'class':'table'})
    paper['tables'] = get_biorxiv_tables(tables_html, content_url)

    # get figures
    figures_html = soup.find_all('div',attrs={'class':'fig'})
    

    paper['figures'] = get_biorxiv_figures(figures_html)

    # get references
    references_html = soup.find('div',attrs={'class':'section ref-list'}).find_all('li')
    paper['references'] = get_biorxiv_references(references_html)
    
    return paper

## Reading from json function

In [261]:
def ref_to_str(reference):
    ''' Convert reference to string '''
    output = ''
    if reference['label'] != '':
        output = reference['label'] + ' - '
    if reference['title'] != '':
        output += reference['title'] + ', '
    if reference['authors'] != []:
        output += ', '.join(reference['authors'])
    if reference['date'] != None:
        output += reference['date']
    return output

def tab_to_str(table, indent=''):
    ''' Convert table to string '''
    output = '\n'
    output += indent + '-'*100 + '\n'
    output += indent + table['id'] + ' | ' + table['label'] + '\n'
    output += indent + table['img'] + '\n'
    output += indent + table['description']
    output += indent + '-'*100
    output += '\n'
    return output

def fig_to_str(figure, indent=''):
    ''' Convert figure to string '''
    output = '\n'
    output += indent + '-'*100 + '\n'
    output += indent + figure['id'] + ' | ' + figure['label'] + '\n'
    output += indent + figure['img'] + '\n'
    output += indent + figure['description']
    output += indent + '-'*100
    output += '\n'
    return output

In [262]:
def paper_to_string(paper, appendix=False):
    ''' Converts a paper to string formart '''
    output = '({} | {} | {})\n'.format(paper['source'], paper['id'], paper['url'])
    output += paper['title'] + '\n'
    output += ', '.join(paper['authors']) + '\n'
    for section in paper['sections']:
        if (not appendix) and ('appendix' in section['title'].lower()):
            pass
        else:
            output += section_to_str(section, paper)
            output += '\n'
    output += '\n\n'.join([ref_to_str(ref) for ref in paper['references']])    
    return output

In [263]:
def section_to_str(section, paper, depth=0):
    indent = '  '*depth
    output = indent + section['title']
    for content in section['content']:
        if type(content) == dict:
            if 'title' in content.keys():
                # is section = explore further 
                output += '\n'
                output += section_to_str(content, paper, depth+1)
            elif 'id' in content.keys():
                # is fig or table, get string
                if content['type'] == 'table':
                    output += '\n'
                    output += tab_to_str(paper['tables'][content['id']], indent) # content['id']
                    # insert table description from paper['tables']
                elif content['type'] == 'figure':
                    output += ' \n'
                    output += fig_to_str(paper['figures'][content['id']], indent) # content['id']
                    # insert figure description from paper['figures']
                else:
                    print('content not recognized: {}'.format(content))
        elif type(content) == str:
            output += '\n' + indent + content
    return output

# Test Process

Test for a single file

## Scraping

In [264]:
# init session
session = requests.session()

# load metadata
df_meta = pd.read_csv('metadata.csv')

papers_df = df_meta[df_meta['source_x']=='biorxiv'][['sha','doi']]
papers_array = papers_df.to_numpy()

row = papers_array[0]
print(row)

['f056da9c64fbf00a4645ae326e8a4339d015d155' '10.1101/001727']


In [265]:
paper = {}
paper['source'] = 'biorxiv'
paper['id'] = row[0]
paper['url'] = 'https://biorxiv.org/content/' + row[1].strip('doi.org') + 'v1.full'

In [266]:
print('Fetching paper data at {}'.format(paper['url']))
html = session.get(paper['url'])
tmp_soup = BeautifulSoup(html.content,'html.parser')

Fetching paper data at https://biorxiv.org/content/10.1101/001727v1.full


In [267]:
paper_webdata = biorxiv_html_paper_data_extractor(tmp_soup)
paper = dict(chain(paper.items(),paper_webdata.items()))

In [268]:
# save to file 
with open('data/biorxiv/{}.json'.format(paper['id']), 'w') as fp:
    json.dump(paper, fp, indent=4)

## Reading

In [5]:
existing_papers = get_file_names('data/biorxiv')
print(existing_papers[0])

test_paper = Paper('biorxiv')
test_paper.import_json('data/biorxiv/{}'.format(existing_papers[0]))

f056da9c64fbf00a4645ae326e8a4339d015d155.json


True

In [6]:
print(test_paper.to_str())

(biorxiv | f056da9c64fbf00a4645ae326e8a4339d015d155 | https://biorxiv.org/content/10.1101/001727v1.full)
SIANN: Strain Identification by Alignment to Near Neighbors
Samuel S. Minot, Stephen D. Turner, Krista L. Ternus, Dana R. Kadavy
Abstract
Next-generation sequencing is increasingly being used to study samples composed of mixtures of organisms, such as in clinical applications where the presence of a pathogen at very low abundance may be highly important. We present an analytical method (SIANN: Strain Identification by Alignment to Near Neighbors) specifically designed to rapidly detect a set of target organisms in mixed samples that achieves a high degree of species- and strain-specificity by aligning short sequence reads to the genomes of near neighbor organisms, as well as that of the target. Empirical benchmarking alongside the current state-of-the-art methods shows an extremely high Positive Predictive Value, even at very low abundances of the target organism in a mixed sample. 

## Analyzing

In a research paper, all sections are not equal. The Abstract should contain the most important words and the Results or Discussion should have the most interesting aspects. 
Let's analyze a paper for each of it's sections.

In [7]:
import spacy

nlp = spacy.load('en_core_web_sm')

stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [34]:
from collections import Counter

In [30]:
for section in test_paper.paper['sections']:
    print(section['title'])

Abstract
Introduction
Methods
Results
Discussion
Appendix 1: Target Pathogen Database
Appendix 2: Viral Database
Appendix 3: Bacterial Database


In [32]:
test_paper.paper['sections'][6]['content']

['Abaca bunchy top virus DNA-C',
 'Abaca bunchy top virus DNA-M',
 'Abaca bunchy top virus DNA-N',
 'Abaca bunchy top virus DNA-R',
 'Abaca bunchy top virus DNA-S',
 'Abaca bunchy top virus segment 2',
 'Abalone shriveling syndrome-associated virus',
 'Abutilon Brazil virus DNA A',
 'Abutilon Brazil virus DNA B',
 'Abutilon mosaic virus DNA A',
 'Abutilon mosaic virus DNA B',
 'Acanthocystis turfacea Chlorella virus 1',
 'Acheta domesticus densovirus',
 'Acholeplasma phage L2',
 'Acholeplasma phage MV-L1',
 'Acidianus bottle-shaped virus',
 'Acidianus filamentous virus 1',
 'Acidianus filamentous virus 2',
 'Acidianus filamentous virus 3',
 'Acidianus filamentous virus 6',
 'Acidianus filamentous virus 7',
 'Acidianus filamentous virus 8',
 'Acidianus filamentous virus 9',
 'Acidianus rod-shaped virus 1',
 'Acidianus spindle-shaped virus 1',
 'Acidianus two-tailed virus',
 'Actinomyces phage Av-1',
 'Actinoplanes phage phiAsp2',
 'Acyrthosiphon pisum bacteriophage APSE-1',
 'Adeno-asso

In [10]:
abstract_text = ' '.join(test_paper.paper['sections'][0]['content'])
abstract_doc = nlp(abstract_text)

In [14]:
lemmas = [token.lemma_ for token in abstract_doc]

In [38]:
a_lemmas = [lemma.lower() for lemma in lemmas if lemma.isalpha() and lemma not in stopwords]

In [39]:
' '.join(a_lemmas)

'generation sequencing increasingly use study sample compose mixture organism clinical application presence pathogen low abundance highly important present analytical method siann strain identification alignment near neighbors specifically design rapidly detect set target organism mixed sample achieve high degree strain specificity align short sequence read genome near neighbor organism target empirical benchmarke alongside current state art method extremely high positive predictive value low abundance target organism mixed sample siann available illumina basespace app signature science llc siann result present streamlined report design comprehensible non specialist user provide powerful tool rapid specie detection mixed sample focus set customizable target organism near neighbor siann operate quickly low computational requirement deliver highly accurate result'

In [48]:
word_count = Counter(a_lemmas)
word_count.most_common(5)

[('organism', 5), ('sample', 4), ('siann', 4), ('target', 4), ('low', 3)]

In [17]:
# Named Entity Recognition test
ne = [(ent.text, ent.label_) for ent in abstract_doc.ents]
print(ne)

[('Positive Predictive Value', 'ORG'), ('Illumina BaseSpace', 'ORG'), ('Signature Science', 'WORK_OF_ART')]


In [16]:
text = ' '.join(test_paper.paper['sections'][0]['content'])
print(text)

Next-generation sequencing is increasingly being used to study samples composed of mixtures of organisms, such as in clinical applications where the presence of a pathogen at very low abundance may be highly important. We present an analytical method (SIANN: Strain Identification by Alignment to Near Neighbors) specifically designed to rapidly detect a set of target organisms in mixed samples that achieves a high degree of species- and strain-specificity by aligning short sequence reads to the genomes of near neighbor organisms, as well as that of the target. Empirical benchmarking alongside the current state-of-the-art methods shows an extremely high Positive Predictive Value, even at very low abundances of the target organism in a mixed sample. SIANN is available as an Illumina BaseSpace app, as well as through Signature Science, LLC. SIANN results are presented in a streamlined report designed to be comprehensible to the non-specialist user, providing a powerful tool for rapid speci

In [56]:
def get_section_text(section):
    ''' Returns a string containing only paragraph text of sections '''
    text = ''
    for content in section['content']:
        if type(content) == str:
            text += ' ' + content
        elif type(content) == dict and 'title' in content.keys():
            text += ' ' + get_section_text(content)
    return text

In [58]:
for section in test_paper.paper['sections']:
    if 'Appendix' not in section['title']:
        print(section['title'])
        # process section content
        text = get_section_text(section)
        doc  = nlp(text)
        lemmas = [token.lemma_ for token in doc]
        a_lemmas = [lemma.lower() for lemma in lemmas if lemma.isalpha() and lemma not in stopwords]

        # get top 5 most frequent words 
        word_count = Counter(a_lemmas)
        print(word_count.most_common(5))

        # test NER on content 
        ne = [(ent.text, ent.label_) for ent in doc.ents]
        print('NER:')
        print(ne)

        print('-'*60)

Abstract
[('organism', 5), ('sample', 4), ('siann', 4), ('target', 4), ('low', 3)]
NER:
[('Positive Predictive Value', 'ORG'), ('Illumina BaseSpace', 'ORG'), ('Signature Science', 'WORK_OF_ART')]
------------------------------------------------------------
Introduction
[('organism', 7), ('sample', 5), ('genome', 5), ('sequence', 5), ('present', 4)]
NER:
[('one', 'CARDINAL'), ('one', 'CARDINAL'), ('one', 'CARDINAL'), ('millions', 'CARDINAL'), ('2013', 'DATE'), ('hundreds', 'CARDINAL'), ('ref-15,"Wu', 'PRODUCT'), ('al.', 'GPE'), ('2011', 'DATE'), ('2013', 'DATE'), ('second', 'ORDINAL')]
------------------------------------------------------------
Methods
[('database', 26), ('specie', 24), ('organism', 22), ('read', 17), ('method', 16)]
NER:
[('2012', 'DATE'), ('SIANN', 'PRODUCT'), ('SIANN', 'PRODUCT'), ('& Salzberg', 'GPE'), ('2012', 'DATE'), ('algorithm', 'ORG'), ('ref-12,"Segata', 'LAW'), ('al.', 'GPE'), ('2012', 'DATE'), ('two', 'CARDINAL'), ('only two', 'CARDINAL'), ('one', 'CARDINAL

In [141]:
# build keyterms database from key terms in ARTICLES
key_terms = { 'SIANN':{'type':'method',
                       'def':'Strain Identification by Alignment to Near Neighbors',
                       'from':test_paper.paper['id']}
            }

for section in test_paper.paper['sections']:
    if 'Appendix 1' in section['title']:
        print(section['title'])
        for content in section['content']:
            key_terms[content] = {'type':'target pathogen','from':test_paper.paper['id']}
    elif 'Appendix 2' in section['title']:
        print(section['title'])
        for content in section['content']:
            key_terms[content] = {'type':'viral','from':test_paper.paper['id']}
    elif 'Appendix 3' in section['title']:
        print(section['title'])
        for content in section['content']:
            key_terms[content] = {'type':'bacterial','from':test_paper.paper['id']}

In [ ]:
key_terms

### HTML Display

Objective: display text with special formating.

In [22]:
from IPython.core.display import display, HTML

In [25]:
def text_to_html(text, words=[]):
    ''' Replaces words in text with html tags that highlight them '''
    html = '<p>{}</p>'.format(text)
    if len(words) > 0:
        for word in words:
            html_word = '<strong>{}</strong>'.format(word)
            html = html.replace(word, html_word)
    return html

In [27]:
html_text = text_to_html(abstract_text, a_lemmas)
display(HTML(html_text))

## Visualizing

In [119]:
def authors_to_html(authors):
    ''' Converts list of authors to html '''
    html = ''
    html += ', '.join(['<span class="author">{}</span>'.format(author) for author in authors])
    return html

def graph_to_html(graph):
    ''' Converts graphic element to html '''
    html = '<table id={} class="figure"><tr><td>'.format(graph['id'])
    html += '<tr><td><img src={} alt={}></tr></td>'.format(graph['img'],graph['label'])
    html += '<tr><td><b>{}</b></tr></td>'.format(graph['label'])
    html += '<tr><td><span class="description">{}</span></td></tr>'.format(graph['description'])
    html += '</table>'
    return html

def section_to_html(section, paper):
    ''' Converts a section to html '''
    html = '<table>'
    html += '<tr><td>{}</td></tr>'.format(section['title'])
    for content in section['content']:
        html += '<tr>'
        if type(content) == str:
            html += '<td><p>{}</p></td>'.format(content)
        elif type(content) == dict:
            if 'title' in content.keys():
                # subsection
                html += '<td>{}</td>'.format(section_to_html(content, paper))
            elif 'id' in content.keys():
                # graphic element
                graph_html = graph_to_html(paper[content['type'] + 's'][content['id']])
                html += '<td>{}</td>'.format(graph_html)
        html += '</tr>'
    html += '</table>'
    return html

def ref_to_html(ref):
    ''' Converts a reference to html '''
    html = '<span class="reference">'
    if ref['label'] != '':
        html += ref['label'] + ' - '
    html += ref['title'] + ', '
    html += authors_to_html(ref['authors']) + ', '
    if ref['date']:
        html += ref['date'] + ', '
    if ref['links']:
        html += ', '.join(['<a href="{}">{}</a>'.format(link[1],link[0]) for link in ref['links']])
    html += '</span>'
    return html

def refs_to_html(refs):
    ''' Converts all references to html '''
    refs_html = '<table id="references" class="section">'
    refs_html += '<tr><td><h2>References</h2></td></tr>'
    for ref in refs:
        refs_html += '<tr><td>'
        refs_html += ref_to_html(ref)
        refs_html += '</td></tr>'
    refs_html += '</table>'
    return refs_html

def paper_to_html(paper):
    ''' Converts a json paper to HTML format '''
    html = '<table>'
    html += '<tr><td style="text-align:left;">{}</td></tr>'.format(paper['title'])
    html += '<tr><td>{}</td></tr>'.format(authors_to_html(paper['authors']))
    for section in paper['sections']:
        if 'Appendix' not in section['title']:
            html += '<tr><td>{}</td></tr>'.format(section_to_html(section, paper))
    html += '<tr><td>{}</td></tr>'.format(refs_to_html(paper['references']))
    html += '</table>'
    return html

In [124]:
css= '''<style type="text/css">
    .article {
      width: 100%;
    }

    .section {
      border-top: 3px solid black;
      width: 100%;
    }

    #title {
      text-align: center;
    }

    .section td {
      text-align: left;
    }

    .subtitle {
      font-weight: bold;
    }

    img {
      width: 50%;
      height: 50%;
    }

    .figure {
      border: 1px solid black;
      vertical-align: center;
    }
  </style>'''

In [125]:
paper_html = paper_to_html(test_paper.paper)
display(HTML(css))
display(HTML(paper_html))

SIANN: Strain Identification by Alignment to Near Neighbors
"Samuel S. Minot, Stephen D. Turner, Krista L. Ternus, Dana R. Kadavy"
Abstract
"Next-generation sequencing is increasingly being used to study samples composed of mixtures of organisms, such as in clinical applications where the presence of a pathogen at very low abundance may be highly important. We present an analytical method (SIANN: Strain Identification by Alignment to Near Neighbors) specifically designed to rapidly detect a set of target organisms in mixed samples that achieves a high degree of species- and strain-specificity by aligning short sequence reads to the genomes of near neighbor organisms, as well as that of the target. Empirical benchmarking alongside the current state-of-the-art methods shows an extremely high Positive Predictive Value, even at very low abundances of the target organism in a mixed sample. SIANN is available as an Illumina BaseSpace app, as well as through Signature Science, LLC. SIANN results are presented in a streamlined report designed to be comprehensible to the non-specialist user, providing a powerful tool for rapid species detection in a mixed sample. By focusing on a set of (customizable) target organisms and their near neighbors, SIANN can operate quickly and with low computational requirements while delivering highly accurate results."
Introduction
"There are many different methods that characterize the mixture of organisms present within a metagenomic dataset. Such datasets are generated when a complex environmental sample is processed by a “next-generation” high-throughput genome sequencing protocol, and they consist of large numbers of short nucleotide sequences. Each sequence represents a small fragment of a randomly selected genome from the very large collection of genomes present in the source sample. Those sequences indicate the presence of one organism or another according to their similarity to a set of known reference genomes. While a given sequence may be unique to one species, it also may be found in diverse organisms across the tree of life. Therefore, one analytical challenge (among many) is to take that collection of sequences (likely numbering in the millions) and accurately determine what species are present in the sample. Here we describe a novel method (SIANN: Strain Identification by Alignment to Near Neighbors) that is specifically designed to rapidly detect a set of targeted organisms from a metagenomic dataset by aligning reads to genomic regions that are unique at the strain or species level."
"The analytical question motivating a particular piece of metagenomic bioinformatic analysis may vary widely by user and sample type ([ref-11,""Segata, et al., 2013""]). For example, the function of the human gut microbiome may depend on the relative abundance of hundreds of species of bacteria and the types of metabolic genes they contain ([ref-15,""Wu, et al., 2011""]; [ref-10,""Schloissnig, et al., 2013""]). In contrast, the clinical treatment of a patient may depend on whether or not a particular virus, or a consortium of co-infecting pathogens, is/are detected in their blood. It is this second class of presence/absence questions that SIANN is designed to address. SIANN is appropriate for situations in which a user wants to know whether a particular organism or set of organisms is present in a sample, but isn’t interested in the functions encoded in their genomes, the relative abundance of each organism, or any other more in-depth analysis."
Methods
Approach
"Metagenomic classification methods are based on a wide variety of theoretical underpinnings. The basic varieties include alignment of reads to various nucleotide databases or exact matching to nucleotide or protein signature sequences (or kmers). A representative set of recent methods are described in [T1,""Table 1""] (also see [ref-2,""Bazinet & Cummings 2012""])."
""


In [121]:
paper_html

'<table><tr><td style="text-align:left;">SIANN: Strain Identification by Alignment to Near Neighbors</td></tr><tr><td><span class="author">Samuel S. Minot</span>, <span class="author">Stephen D. Turner</span>, <span class="author">Krista L. Ternus</span>, <span class="author">Dana R. Kadavy</span></td></tr><tr><td><table><tr><td>Abstract</td></tr><tr><td><p>Next-generation sequencing is increasingly being used to study samples composed of mixtures of organisms, such as in clinical applications where the presence of a pathogen at very low abundance may be highly important. We present an analytical method (SIANN: Strain Identification by Alignment to Near Neighbors) specifically designed to rapidly detect a set of target organisms in mixed samples that achieves a high degree of species- and strain-specificity by aligning short sequence reads to the genomes of near neighbor organisms, as well as that of the target. Empirical benchmarking alongside the current state-of-the-art methods show

# Get All Biorxiv Papers

In [ ]:
source = 'biorxiv'
print("\nThis script only scrapes articles from {}, if not already downloaded.".format(source))

session = requests.session()

# import list of articles in data folder
existing_shas = [name.strip('.json') for name in get_file_names('data/{}'.format(source))]

# import metadata dataframe
df_meta = pd.read_csv('metadata.csv')

'''
sources = df_meta['source_x'].unique()
for source in sources:
    papers_df = df_meta[df_meta['source_x']==source][['sha','doi']]
    papers_array = papers_df.to_numpy()
    print('There are {} papers ids from {}.'.format(len(papers_array), source))
'''

missing_articles = df_meta[(df_meta['source_x']=='biorxiv')&(~df_meta['sha'].isin(existing_shas))]
print("{} articles to scrape".format(missing_articles.shape[0]))

papers_df = missing_articles[['sha','doi']]
papers_array = papers_df.to_numpy()
indexes = papers_df.index.to_numpy()

In [53]:
# get papers from data/biorxiv
existing_paper_ids = set([name.strip('.json') for name in get_file_names('data/biorxiv')])

# go through all papers
# new_paper_cnt = 0
for row in papers_array:
    if row[0] not in existing_paper_ids:
        paper = {}
        paper['source'] = 'biorxiv'
        paper['id'] = row[0]
        paper['url'] = 'https://biorxiv.org/content/' + row[1].strip('doi.org') + 'v1.full'
        print('Fetching paper data at {}'.format(paper['url']))
        html = session.get(paper['url'])
        tmp_soup = BeautifulSoup(html.content,'html.parser')
        paper_webdata = biorxiv_html_paper_data_extractor(tmp_soup)
        paper = dict(chain(paper.items(),paper_webdata.items()))    
        with open('data/biorxiv/{}.json'.format(paper['id']), 'w') as fp:
            json.dump(paper, fp, indent=4)
        print('Paper ID: {} saved to data/biorxiv.'.format(paper['id']))
        new_paper_cnt += 1
        print('-'*60)
print('Added {} papers to data/biorxiv.'.format(new_paper_cnt))

Fetching paper data at https://biorxiv.org/content/10.1101/010389v1.full
Paper ID: eccef80cfbe078235df22398f195d5db462d8000 saved to data/biorxiv.
------------------------------------------------------------
Fetching paper data at https://biorxiv.org/content/10.1101/012070v1.full
Paper ID: c41fdb2efd6d61384a92a84cbba3f8233629a41b saved to data/biorxiv.
------------------------------------------------------------
Fetching paper data at https://biorxiv.org/content/10.1101/018481v1.full
Paper ID: 33565294e6bc67fb7ee14dcae6cfdb08148f4ea5 saved to data/biorxiv.
------------------------------------------------------------
Fetching paper data at https://biorxiv.org/content/10.1101/027722v1.full
Paper ID: 1f9d3f9a1a0e8db6a086e0a2b5ba50cf9f235dae saved to data/biorxiv.
------------------------------------------------------------
Fetching paper data at https://biorxiv.org/content/10.1101/029397v1.full
Paper ID: 01e3b313e78a352593be2ff64927192af66619b5 saved to data/biorxiv.
---------------------

AttributeError: 'NoneType' object has no attribute 'text'

In [41]:
url = 'https://biorxiv.org/content/10.1101/010389v1.full'
html = session.get(url)
tmp_soup = BeautifulSoup(html.content,'html.parser')

In [51]:
sections = {}
for section in tmp_soup.find_all('div',attrs={'class':'section'}):
    section_header = section.find('h2')
    if section_header:
        section_name = section_header.text
    
    if section_header and section_name != 'References' and section_name != 'Acknowledgments':
        #print(section_name)
        sections[section_name] = ''
        for p in section.find_all('p'):
            sections[section_name] += p.text + '\n'
        #print(sections[section_name])
        #print()
print(sections)

{'Abstract': 'Background Developing methods to reconstruct transmission histories for viral outbreaks could provide critical information to support locating sources of disease transmission. Phylogenetic methods used to measure the degree of relatedness among sequenced viral samples have proven useful in identifying potential outbreak sources. The complex nature of infectious disease, however, makes it difficult to assign a rigorously defined quantitative confidence value assessing the likelihood of a true direct transmission event using genetic data alone.\nResults A new method is presented to calculate a confidence value assessing the likelihood of a transmission event using both phylogenetic inference and limited knowledge of incubation and infectious duration times. The method is applied to simulations of a foot and mouth disease (FMD) outbreak to demonstrate how the combination of both phylogenetic and epidemiology data can be used to strengthen the assessment of the likelihood of 

In [18]:
# get tables
tables = []
html_tables = tmp_soup.find_all('div',attrs={'class':'table'})
for html_table in html_tables:
    table = {}
    table['id'] = html_table['id']
    label = html_table.find('span',attrs={'class':'table-label'})
    if label:
        table['label'] = label.text
    caption = html_table.find('span',attrs={'class':'caption-title'})
    if caption:
        table['caption'] = caption.text
    table['description'] = ''
    for p in html_table.find_all('p'):
        table['description'] += p.text + '\n'
    table['image'] = None
    tables.append(table)

for table in tables:
    print(table)

{'id': 'T1', 'label': 'Table 1.', 'description': 'Summary of methods for metagenomic classification.\n', 'image': None}
{'id': 'T2', 'label': 'Table 2.', 'caption': 'The abundance of each target organism in each set of simulated datasets. Each set is indicated by the number in the top row, and was generated with 50 replicates.', 'description': '', 'image': None}


In [231]:
html_figures[0].find_all('p')

[<p class="first-child" id="p-9">A) For a group of strains belonging to two different species, some regions may be unique to each species (region 1), while other regions may be unique to strains within each species (regions 2 and 3). B) A set of reads are aligned to these genomes, and the ones that align in a species- or strain-specific manner are identified by the combination of genomes to which they align. In this example, Strain B of Species I is the organism identified.</p>]

# Test

In [231]:
# keep track of index
i = 0
modified = False
for row in papers_array:
    paper = {}
    paper['source'] = source
    # if no sha => generate one that doesn't exist.
    if str(row[0]) == 'nan':
        paper['id'] = generate_sha(existing_shas)
        # save new hash and add to existing_shas
        existing_shas.append(paper['id'])
        df_meta.loc[indexes[i], 'sha'] = paper['id']
        modified = True
    else:
        paper['id'] = row[0]
    paper['url'] = 'https://biorxiv.org/content/' + row[1].strip('doi.org') + 'v1.full'
    if source == 'biorxiv':
        paper = scrape_biorxiv(paper)
    else:
        print('Error source not biorxiv. ({})'.format(source))
    print(paper['title'],paper['id'])
    '''
    with open('data/{}/{}.json'.format(source,paper['id'])):
        json.dump(paper, fp, indent=4)
    '''
    print('Paper ID: {} saved to data/{}'.format(paper['id'], source))
    print('-'*60)
    i += 1

# if dataframe was changed, save it to csv
if modified:
    df_meta.to_csv('new_metadata.csv')

Fetching paper data at https://biorxiv.org/content/10.1101/030742v1.full
What’s in my pot? Real-time species identification on the MinION™ 
Paper ID:  saved to data/biorxiv
------------------------------------------------------------
Fetching paper data at https://biorxiv.org/content/10.1101/060434v1.full
The ATP synthase subunit β (ATP5B) is an entry factor for the hepatitis E virus 52a98150f5d9e362964cbaaaab7921df
Paper ID: 52a98150f5d9e362964cbaaaab7921df saved to data/biorxiv
------------------------------------------------------------
Fetching paper data at https://biorxiv.org/content/10.1101/073098v1.full
Zika virus infection as a cause of congenital brain abnormalities and Guillain-Barré syndrome: systematic review 46920aca40b20654ab2ca3372206bc2f
Paper ID: 46920aca40b20654ab2ca3372206bc2f saved to data/biorxiv
------------------------------------------------------------
Fetching paper data at https://biorxiv.org/content/10.1101/074559v1.full
Identification of quercetin from fru

KeyboardInterrupt: 